prototype notebook to get a DMPNN running; loosely based on run_training.py

implementation:
- fff
- fff
- fff

In [129]:
import csv
import os
import sys
import numpy as np
import torch
import pickle

from logging import Logger
from typing import List
from tqdm import trange

from torch.optim.lr_scheduler import ExponentialLR
from torch_geometric.datasets import QM9

In [130]:
# cd to chempropBayes
%cd /Users/georgelamb/Documents/GitHub/chempropBayes

/Users/georgelamb/Documents/GitHub/chempropBayes


In [131]:
# import from chempropBayes
from chemprop.trainBayes.evaluate import evaluate, evaluate_predictions
from chemprop.trainBayes.predict import predict
from chemprop.trainBayes.train import train
from chemprop.args import TrainArgs
from chemprop.data import StandardScaler, MoleculeDataLoader
from chemprop.data.utils import get_class_sizes, get_data, get_task_names, split_data
from chemprop.models import MoleculeModel
from chemprop.nn_utils import param_count
from chemprop.utils import build_optimizer, build_lr_scheduler, get_loss_func, get_metric_func, load_checkpoint,\
    makedirs, save_checkpoint, save_smiles_splits

In [132]:
# instantiate args object
args = TrainArgs

In [133]:
# set pytorch seed for random initial weights
args.pytorch_seed = 0
torch.manual_seed(args.pytorch_seed);

In [134]:
# get data (QM9 csv -> MoleculeDataset)
args.data_path = '/Users/georgelamb/Documents/GitHub/chempropBayes/data/QM9.csv' # path to csv
args.target_columns = ['mu'] # names of columns containing target values
args.task_names = args.target_columns # set task name to 'mu'
args.max_data_size = 2000 # number of data points to load

data = get_data(path=args.data_path, args=args)

args.num_tasks = data.num_tasks()
args.features_size = data.features_size()

0it [00:00, ?it/s]
100%|██████████| 2000/2000 [00:00<00:00, 17936.72it/s]


In [135]:
# split data
train_data, val_data, test_data = split_data(data=data, split_type=args.split_type, sizes=args.split_sizes, 
                                             seed=args.seed, args=args, logger=None)


In [136]:
# features scaling
args.features_scaling = True
if args.features_scaling:
    features_scaler = train_data.normalize_features(replace_nan_token=0)
    val_data.normalize_features(features_scaler)
    test_data.normalize_features(features_scaler)
else:
    features_scaler = None

In [138]:
# initialize scaler and scale training targets by subtracting mean and dividing standard deviation (regression only)
args.dataset_type = 'regression'
if args.dataset_type == 'regression':
    train_smiles, train_targets = train_data.smiles(), train_data.targets()
    scaler = StandardScaler().fit(train_targets)
    scaled_targets = scaler.transform(train_targets).tolist()
    train_data.set_targets(scaled_targets)
else:
    scaler = None

In [140]:
# get loss and metric functions
args.metric = 'rmse'
loss_func = get_loss_func(args)
metric_func = get_metric_func(metric=args.metric)

In [142]:
# set up test set evaluation
test_smiles, test_targets = test_data.smiles(), test_data.targets()
if args.dataset_type == 'multiclass':
    sum_test_preds = np.zeros((len(test_smiles), args.num_tasks, args.multiclass_num_classes))
else:
    sum_test_preds = np.zeros((len(test_smiles), args.num_tasks))

In [147]:
# create data loaders
train_data_loader = MoleculeDataLoader(
    dataset=train_data,
    batch_size=args.batch_size,
    class_balance=args.class_balance,
    shuffle=True,
    seed=args.seed
)
val_data_loader = MoleculeDataLoader(
    dataset=val_data,
    batch_size=args.batch_size,
)
test_data_loader = MoleculeDataLoader(
    dataset=test_data,
    batch_size=args.batch_size,
)

In [149]:
# load/build model
args.features_size = 0
model = MoleculeModel(args)

TypeError: new() received an invalid combination of arguments - got (NoneType, int), but expected one of:
 * (*, torch.device device)
      didn't match because some of the arguments have invalid types: ([31;1mNoneType[0m, [31;1mint[0m)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [ ]:
# optimisers

In [ ]:
# learning rate schedulers

In [ ]:
# run training

In [ ]:
# evaluate on test set